In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from statistics import mean
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix

## Loading dataset

In [44]:
x_train = pd.read_pickle("training/x_train.pkl")
y_train = pd.read_pickle("training/y_train.pkl")
x_test = pd.read_pickle("training/x_test.pkl")
y_test = pd.read_pickle("training/y_test.pkl")

In [48]:
print(x_train.columns[0:15])
x_train.columns


Index(['user', 'x_len', 'y_len', 'avg_ps', 'ratio', 'dif_pen', 'dif_az',
       'pen_up', 'sign_time', 'x_speed', 'y_speed', 'x_size', 'total_length',
       'tx0', 'ty0'],
      dtype='object')


Index(['user', 'x_len', 'y_len', 'avg_ps', 'ratio', 'dif_pen', 'dif_az',
       'pen_up', 'sign_time', 'x_speed',
       ...
       'tx98', 'ty98', 'tp98', 'ta98', 'te98', 'tx99', 'ty99', 'tp99', 'ta99',
       'te99'],
      dtype='object', length=513)

In [4]:
# xt = torch.tensor(X_train.values)
# yt = torch.tensor(Y_train.values)

In [5]:
# dataset = TensorDataset(xt, yt)

In [6]:
# train_dl = DataLoader(dataset, BATCH_SIZE, shuffle=False)

In [49]:
print(x_train)

      user   x_len   y_len         avg_ps     ratio  dif_pen  dif_az  pen_up  \
3062    31  1600.0  1135.0     440.445860  1.409692        9      20      30   
4441    86  2818.0  1549.0     437.117845  1.819238       12      31      71   
562     23  4675.0  4734.0     460.980707  0.987537       26      64     138   
512     21  3531.0  1282.0     407.962500  2.754290        6      18       0   
1311    56  2151.0   812.0  491095.406360  2.649015       14      28      94   
40       1  2843.0  1093.0     812.722922  2.601098        7      22      72   
3746    59  3818.0  1067.0  374332.129964  3.578257        8      22      62   
3756    59  2531.0  1185.0     415.081481  2.135865        7      18      44   
44       1  2656.0  1557.0     847.739938  1.705845       18      30      43   
575     24  3819.0  3239.0     261.134367  1.179068       36      44     448   
2274     0  3612.0  1598.0     681.326733  2.260325       15      19       0   
2235    98  2272.0  1074.0     500.08675

## Autoencoder model

In [51]:
def autoencoder():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(input_shape = (513,),activation = 'relu',units = 513),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(200,activation='relu'),
    tf.keras.layers.Dense(300,activation='relu'),
    tf.keras.layers.Dense(400,activation= 'relu'),
    tf.keras.layers.Dense(513,activation = 'relu')
  ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.MeanSquaredLogarithmicError(),
                metrics=['accuracy'])
    model.summary()
    return model

In [52]:
auto_model = autoencoder()
historty = auto_model.fit(x_train.values,x_train.values,epochs=120,validation_data=(x_test.values[:249], x_test.values[:249]))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_99 (Dense)             (None, 513)               263682    
_________________________________________________________________
dense_100 (Dense)            (None, 400)               205600    
_________________________________________________________________
dense_101 (Dense)            (None, 300)               120300    
_________________________________________________________________
dense_102 (Dense)            (None, 200)               60200     
_________________________________________________________________
dense_103 (Dense)            (None, 100)               20100     
_________________________________________________________________
dense_104 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_105 (Dense)            (None, 200)              

Epoch 99/120
3337/3337 [==============================] - 1s 429us/sample - loss: 2.2642 - accuracy: 0.9308 - val_loss: 2.1866 - val_accuracy: 0.9438
Epoch 100/120
3337/3337 [==============================] - 1s 427us/sample - loss: 2.3869 - accuracy: 0.9311 - val_loss: 2.2871 - val_accuracy: 0.9438
Epoch 101/120
3337/3337 [==============================] - 2s 482us/sample - loss: 2.2984 - accuracy: 0.9311 - val_loss: 2.1966 - val_accuracy: 0.9438
Epoch 102/120
3337/3337 [==============================] - 2s 486us/sample - loss: 2.2889 - accuracy: 0.9311 - val_loss: 2.4451 - val_accuracy: 0.9438
Epoch 103/120
3337/3337 [==============================] - 1s 429us/sample - loss: 2.2980 - accuracy: 0.9308 - val_loss: 2.2033 - val_accuracy: 0.9438
Epoch 104/120
3337/3337 [==============================] - 2s 500us/sample - loss: 2.2792 - accuracy: 0.9311 - val_loss: 2.1799 - val_accuracy: 0.9438
Epoch 105/120
3337/3337 [==============================] - 1s 440us/sample - loss: 2.3064 - acc

In [55]:
x_train.shape

(3337, 513)

In [54]:
results = auto_model.evaluate(x_test, x_test)

1431/1431 [==============================] - 0s 126us/sample - loss: 2.2479 - accuracy: 0.9364


In [17]:
pred = auto_model.predict(x_test[1:2])

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
input_s= Input(shape=(513,))

Using TensorFlow backend.


In [36]:
encoded = Dense(units=400, activation='relu')(input_s)
encoded = Dense(units=300, activation='relu')(encoded)
encoded = Dense(units=200, activation='relu')(encoded)
# encoded = Dense(units=100, activation='relu')(encoded)
# decoded = Dense(units=200, activation='relu')(encoded)
decoded = Dense(units=300, activation='relu')(encoded)
decoded = Dense(units=400, activation='relu')(decoded)
decoded = Dense(units=513, activation='relu')(decoded)
autoencoder=Model(input_s, decoded)
encoder = Model(input_s, encoded)
autoencoder.summary()
encoder.summary()
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 513)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 400)               205600    
_________________________________________________________________
dense_33 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_34 (Dense)             (None, 200)               60200     
_________________________________________________________________
dense_35 (Dense)             (None, 300)               60300     
_________________________________________________________________
dense_36 (Dense)             (None, 400)               120400    
_________________________________________________________________
dense_37 (Dense)             (None, 513)               2057

In [37]:
autoencoder.fit(x_train, x_train,
                epochs=100,                
                validation_data=(x_test, x_test))

Train on 3337 samples, validate on 1431 samples
Epoch 1/100
3337/3337 [==============================] - 3s 785us/step - loss: 5.8336 - accuracy: 0.0069 - val_loss: 5.0868 - val_accuracy: 0.0636
Epoch 2/100
3337/3337 [==============================] - 2s 632us/step - loss: 4.9645 - accuracy: 0.5901 - val_loss: 4.8080 - val_accuracy: 0.9364
Epoch 3/100
3337/3337 [==============================] - 2s 670us/step - loss: 4.8089 - accuracy: 0.9299 - val_loss: 4.6208 - val_accuracy: 0.9364
Epoch 4/100
3337/3337 [==============================] - 2s 610us/step - loss: 4.3974 - accuracy: 0.9311 - val_loss: 4.1126 - val_accuracy: 0.9357
Epoch 5/100
3337/3337 [==============================] - 2s 621us/step - loss: 4.0035 - accuracy: 0.9305 - val_loss: 3.8394 - val_accuracy: 0.9364
Epoch 6/100
3337/3337 [==============================] - 2s 624us/step - loss: 3.7806 - accuracy: 0.9308 - val_loss: 3.7367 - val_accuracy: 0.9294
Epoch 7/100
3337/3337 [==============================] - 2s 737us/step

In [38]:
new_features = encoder.predict(x_train.values)

In [39]:
new_features

array([[   0.    ,    0.    ,    0.    , ...,    0.    , 2763.7778,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    , 3746.355 ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    , 3891.8914,
           0.    ],
       ...,
       [   0.    ,    0.    ,    0.    , ...,    0.    , 3039.4053,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    , 3563.9736,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    , 3323.5854,
           0.    ]], dtype=float32)

In [40]:
from sklearn import svm
from sklearn.metrics import accuracy_score

In [41]:
model_svm = svm.SVC()

In [42]:
model_svm.fit(new_features, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [43]:
accuracy_score(y_train[0:100], model_svm.predict(new_features[0:100]))

0.53